Answer all questions and submit them either as an IPython notebook, LaTeX document, or Markdown document. Each question is worth 25 points.

This homework is due Tuesday, October 24, 2017.

In [6]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import pymc3 as pm

# Set seed
np.random.seed(42)

00001	#include <Python.h>
00002	#include "theano_mod_helper.h"
00003	#include "structmember.h"
00004	#include <sys/time.h>
00005	
00006	#if PY_VERSION_HEX >= 0x03000000
00007	#include "numpy/npy_3kcompat.h"
00008	#define PyCObject_AsVoidPtr  NpyCapsule_AsVoidPtr
00009	#define PyCObject_GetDesc  NpyCapsule_GetDesc
00010	#define PyCObject_Check NpyCapsule_Check
00011	#endif
00012	
00013	#ifndef Py_TYPE
00014	#define Py_TYPE(obj) obj->ob_type
00015	#endif
00016	
00017	/**
00018	
00019	TODO: 
00020	- Check max supported depth of recursion
00021	- CLazyLinker should add context information to errors caught during evaluation. Say what node we were on, add the traceback attached to the node.
00022	- Clear containers of fully-useed intermediate results if allow_gc is 1
00023	- Add timers for profiling
00024	- Add support for profiling space used.
00025	
00026	
00027	  */
00028	static double pytime(const struct timeval * tv)
00029	{
00030	  struct timeval t;
00031	  if (!tv)
00032	    {
00033	 

In file included from /Users/ruiwang/.theano/compiledir_Darwin-17.2.0-x86_64-i386-64bit-i386-3.6.2-64/lazylinker_ext/mod.cpp:1:
In file included from /anaconda/envs/myenv/include/python3.6m/Python.h:25:
/anaconda/envs/myenv/bin/../include/c++/v1/stdio.h:108:15: fatal error: 'stdio.h' file not found
#include_next <stdio.h>
              ^~~~~~~~~
1 error generated.




00515	{
00516	  void * ptr_addr = self->thunk_cptr_fn[node_idx];
00517	  int (*fn)(void*) = (int (*)(void*))(ptr_addr);
00518	  if (verbose) fprintf(stderr, "calling non-lazy shortcut (node %i)\n", (int)node_idx);
00519	  int err = 0;
00520	  if (self->do_timing)
00521	    {
00522	      double t0 = pytime(NULL);
00523	      err = fn(self->thunk_cptr_data[node_idx]);
00524	      double t1 = pytime(NULL);
00525	      double ti = PyFloat_AsDouble(PyList_GetItem(self->call_times, node_idx));
00526	      PyList_SetItem(self->call_times, node_idx, PyFloat_FromDouble(t1 - t0 + ti));
00527	      PyObject * count = PyList_GetItem(self->call_counts, node_idx);
00528	      long icount = PyInt_AsLong(count);
00529	      PyList_SetItem(self->call_counts, node_idx, PyInt_FromLong(icount+1));
00530	    }
00531	  else
00532	    {
00533	      err = fn(self->thunk_cptr_data[node_idx]);
00534	    }
00535	
00536	  if (err)
00537	    {
00538	      // cast the argument to a PyList (as described near line 2

AttributeError: module 'theano' has no attribute 'gof'

# Question 1

Epidemiologists are interested in studying the sexual behavior of individuals at risk for HIV infection. Suppose 1500 gay men were surveyed and each was asked how many risky sexual encounters he had in the previous 30 days. Let $n_i$ denote the number of respondents reporting $i$ encounters, for $i = 1, \ldots , 16$. The DataFrame below contains these reponses:

In [ ]:
encounters = pd.DataFrame({'count': np.arange(17),
                          'freq': [379, 299, 222, 145, 109, 95, 73, 59,
                                  45, 30, 24, 12, 4, 2, 0, 1, 1]})

encounters.freq.plot.bar()

These data are poorly fitted by a Poisson model. It is more realistic to assume that the respondents comprise three groups. First, there is a group of people who, for whatever reason, report zero risky encounters even if this is not true. Suppose a respondent has probability $\alpha$ of belonging to this group. 

With probability $\beta$, a respondent belongs to a second group representing typical behavior. Such people respond truthfully, and their numbers of risky encounters are assumed to follow a $\text{Poisson}(\mu)$ distribution. 

Finally, with probability $1 − \alpha − \beta$, a respondent belongs to a high-risk group. Such people respond truthfully, and their numbers of risky encounters are assumed to follow a $\text{Poisson}(\lambda)$ distribution. 

The parameters in the model are $\alpha, \beta, \mu$ and $\lambda$. At the $t$th iteration of EM, we use $\theta^{(t)} = (\alpha^{(t)}, \beta^{(t)}, \mu^{(t)}, \lambda^{(t)})$ to denote the current parameter values. The likelihood of the observed data is given by:

$$L(\theta | n_0, \ldots, n_{16}) \propto \prod_{i=0}^{16} \left[ \frac{\pi_i(\theta)}{i!} \right]^{n_i}$$

where $\pi_i(\theta) = \alpha 1_{(i=0)} + \beta \mu^i \exp(-\mu) + (1-\alpha - \beta) \lambda^i \exp(-\lambda)$.

The observed data are in the `encounters` table above; the *complete* data may be construed to be $(n_{z,0}, n_{t,0}, n_{p,0}), \ldots, (n_{z,16}, n_{t,16}, n_{p,16})$, where $k = z, t, p$ correspond to zero, typical and promiscuous groups, respectively. That is, $n_0 = n_{z,0} + n_{t,0} + n_{p,0}$, and so on. Let $N = \sum_{i=0}^{16} n_i = 1500$.

Also define:

$$\begin{align}
z_0(\theta) &=& \frac{\alpha}{\pi_0(\theta)} \\
t_i(\theta) &=& \frac{\beta \mu^i \exp(-\mu)}{\pi_i(\theta)} \\
p_i(\theta) &=& \frac{(1-\alpha-\beta)\lambda^i \exp(-\lambda)}{\pi_i(\theta)}
\end{align}$$

which correspond to probabilities that respondents with $i$ risky encounters belong to various groups.

a. Show that the EM algorithm provides the following updates:

$$\begin{align}
\alpha^{(t+1)} &=& \frac{n_0 z_0(\theta^{(t)})}{N} \\
\beta^{(t+1)} &=& \sum_i \frac{n_i t_i(\theta^{(t)})}{N} \\
\mu^{(t+1)} &=& \frac{\sum_i i n_i t_i(\theta^{(t)})}{\sum_i n_i t_i(\theta^{(t)})} \\
\lambda^{(t+1)} &=& \frac{\sum_i i n_i p_i(\theta^{(t)})}{\sum_i n_i p_i(\theta^{(t)})}
\end{align}$$


b. Extimate the parameters of the model using the observed data.

In [ ]:
# Write your answer here
def prob(theta, n):
    alpha, beta, mu, lamb = theta
    nlist = np.arange(n)
    return alpha*(nlist == 0)+beta*np.exp(-mu)*np.power(mu, nlist)+(1-alpha-beta)*np.exp(-lamb)*np.power(lamb, nlist)
    
def e_step(theta, nums):
    alpha, beta, mu, lamb = theta
    n = len(nums)
    nlist = np.arange(n)
    probs = prob(theta, n)
    z_0 = alpha/probs[0]
    t = beta*np.exp(-mu)*np.power(mu, nlist)/probs
    p = (1-alpha-beta)*np.exp(-lamb)*np.power(lamb, nlist)/probs
    return z_0, t, p
    
def m_step(theta, z_0, t, p, nums):
    N = np.sum(nums)
    n = len(nums)
    nlist = np.arange(n)
    alpha = nums[0]*z_0/N
    beta = np.sum(nums*t)/N
    mu = np.sum(nlist*nums*t)/np.sum(nums*t)
    lamb = np.sum(nlist*nums*p)/np.sum(nums*p)
    return [alpha, beta, mu, lamb]

def run_em(theta, data):
    
    # Initialize values
    z_0, t, p = 0.0, 0.0, 0.0
    theta_new = [0.0, 0.0, 0.0, 0.0]

    # Stopping criterion
    crit = 1e-5

    # Convergence flag
    converged = False

    # Loop until converged
    while not converged:

        # E-step
        z_0, t, p = e_step(theta, data)
        # M-step
        theta_new = m_step(theta, z_0, t, p, data)

        # Check convergence
        converged = np.all(np.abs((np.array(theta_new) - np.array(theta))) < crit)
        theta = theta_new

    print('The estimated parameters using EM: alpha {0:.4f}, beta {1:.4f}, mu {2:.4f}, lambda {3:.4f}'
          .format(theta_new[0], theta_new[1], theta_new[2], theta_new[3]))

In [ ]:
run_em(theta = [0.1, 0.1, 2, 2], data = np.array(encounters.freq.values))

# Question 2

Suppose $y$ has a binomial distribution with parameters $n$ and $p$, and we are interested in the log-odds value $\theta = \log(p/(1 − p))$. Our prior for $\theta$ is that $\theta \sim N(\mu, \sigma^2)$. It follows that the posterior density of $\theta$ is given, up to a proportionality constant, by:

$$\pi(\theta | y) \propto \frac{\exp(y\theta)}{(1 + exp(\theta))^n} \exp\left[\frac{-(\theta − \mu)^2}{2\sigma^2}\right]$$

For example, suppose we are interested in learning about the probability that a possibly-biased coin lands heads when tossed. *A priori* we believe that the coin is fair, so we assign $\theta$ a $N(0,.25)$ prior. We toss the coin $n = 5$ times and obtain $y = 5$ heads.

1. Using a normal approximation to the posterior density, compute the probability that the coin is biased toward heads (i.e., that θ is positive).
2. Using the prior density as a proposal density, design a rejection algorithm for sampling from the posterior distribution. Using simulated draws from your algorithm, approximate the probability that the coin is biased toward heads.
3. Using the prior density as a proposal density, simulate values from the posterior distribution using the SIR algorithm. Approximate the probability that the coin is biased toward heads.

In [ ]:
# Write your anser here
# question 1
from scipy.optimize import minimize
from scipy.stats import distributions

def cal_logposterior(theta, param):
    y = param[0]
    n = param[1]
    mu = param[2]
    sigma = param[3]
    return -1.0*(y*theta - n*np.log(1+np.exp(theta)) - (theta-mu)**2/(2.0*sigma**2))

param = [5, 5, 0, 0.25]

opt = minimize(cal_logposterior, 0, method='L-BFGS-B', args=(param,))
mode = opt.x
var = opt.hess_inv.todense()
# computer the probability above 0
bias_prob1 = 1 - distributions.norm.cdf(0, loc = mode, scale = np.sqrt(var))
bias_prob1[0][0]

In [ ]:
# question 2
def reject(niter):
    theta = np.random.normal(loc = 0, scale = 0.25, size = niter)
    ratio = np.exp(5*theta)/np.power(1+np.exp(theta), 5)
    return theta[np.random.random(niter) < ratio]

niter = 10**5
part_theta = reject(niter)
plt.hist(part_theta)

acceptance = len(part_theta)
bias_prob2 = np.mean(part_theta > 0)
bias_prob2

In [ ]:
# question 3
niter = 10**5
sir_theta = np.random.normal(loc = 0, scale = 0.25, size = niter)
ratio = np.exp(5*sir_theta)/np.power(1+np.exp(sir_theta), 5)
w = ratio/np.sum(ratio)
theta_post = np.random.choice(sir_theta, size = niter, replace = True, p = w)
plt.hist(theta_post)

bias_prob3 = np.mean(theta_post > 0)
bias_prob3

# Question 3

The goal of this problem is to investigate the role of the proposal distribution in a Metropolis-Hastings algorithm designed to simulate from the posterior distribution of the mixture parameter $\delta$. 

1. Simulate 200 realizations from the mixture distribution:
    $$y_i \sim \delta N(7, 0.5^2) + (1-\delta) N(10, 0.5^2)$$
    with $\delta = 0.7$. Plot a histogram of these data. 
2. Implement a random walk M-H algorithm with proposal $\delta^{\prime} = \delta^{(i)} + \epsilon$ with $\epsilon \sim Unif(-1,1)$. 
3. Reparameterize the problem letting $U = \log\left[\frac{\delta}{1 - \delta}\right]$ and $u^{\prime} = u^{(i)} + \epsilon$. Implement a random walk chain in U-space. 
4. Compare the estimates and convergence behavior of the three algorithms.

In part (1), you are asked to simulate data from a distribution with $\delta$ known. For parts (2)–(3), assume $\delta$ is unknown with prior $\delta \sim Unif( 0,1)$. For parts (2)–(3), provide an appropriate plot and a table summarizing the output of the algorithm. 

To facilitate comparisons, use the same number of iterations, random seed, starting values, and burn-in period for all implementations of the algorithm. 

In [ ]:
# Write answer here
# question 1
np.random.seed(42)

n_sample = 200
delta = 0.7
flag = np.random.uniform(0,1,n_sample) < 0.7
y = flag*np.random.normal(loc=7, scale=0.25, size=n_sample) \
    +(1-flag)*np.random.normal(loc=10, scale=0.25, size=n_sample)
plt.hist(y)

In [ ]:
from scipy.stats import distributions
norm = distributions.norm.pdf

def calc_posterior(delta, y):
    prob = delta*norm(y, loc=7, scale=0.25) + (1-delta)*norm(y, loc=10, scale=0.25)
    return np.sum(np.log(prob))

def metropolis(n_iterations, initial_values, param):               
    # Initialize trace for parameters
    trace = np.empty(n_iterations+1)
    
    # Set initial values
    trace[0] = initial_values
        
    # Calculate joint posterior for initial values
    if param == 'original':
        current_log_prob = calc_posterior(initial_values, y)
    elif param == 'uspace':
        current_log_prob = calc_posterior(np.exp(initial_values)/(1+np.exp(initial_values)), y)
    
    # Initialize acceptance counts
    accepted = 0
    
    for i in range(n_iterations):
    
        if not i%10000: print('Iteration %i' % i)
    
        # Grab current parameter values
        current_params = trace[i]
        
        error = np.random.uniform(-1,1)
        proposed = current_params + error
    
        # Calculate log posterior with proposed value
        if param == 'original':
            proposed_log_prob = calc_posterior(proposed, y)
        elif param == 'uspace':
            proposed_log_prob = calc_posterior(np.exp(proposed)/(1+np.exp(proposed)), y)
        
        # Log-acceptance rate
        alpha = proposed_log_prob - current_log_prob

        # Sample a uniform random variate
        u = np.random.random()

        # Test proposed value
        if np.log(u) < alpha:
            # Accept
            trace[i+1] = proposed
            current_log_prob = proposed_log_prob
            accepted += 1
        else:
            # Reject
            trace[i+1] = trace[i]
    return trace, accepted

In [ ]:
# question 2
trace_1, accepted_1 = metropolis(10**4, 0.5, 'original')
plt.plot(trace_1)
print("The estimated delta is {0: .4f} and the number of accepted is {1: }"
          .format(trace_1[-1], accepted_1))

In [ ]:
# question 3
trace_2, accpeted_2 = metropolis(10**4, 0.5, 'uspace')
plt.plot(trace_2)
print("The estimated delta is {0: .4f} and the number of accepted is {1: }"
          .format(trace_2[-1], accpeted_2))

Question4: We obtain a more acuurate estimated value for $\delta$ but with a low acceptance number. However, the transformation in the U-space allows us to accept at more times than the original model.

# Question 4

Carlin (1992) considers a Bayesian approach to meta-analysis, and includes the following examples of 22 trials of beta-blockers to prevent mortality after myocardial infarction. These data are given below.

In one possible random effects model we assume the true baseline mean (on a log-odds scale) $m_i$ in a trial $i$
is drawn from some population distribution. Let $r^C_i$ denote number of events in the control group in trial $i$, and $r^T_i$ denote events under active treatment in trial $i$. Our model is:

$$\begin{aligned}
r^C_i &\sim \text{Binomial}\left(p^C_i, n^C_i\right) \\
r^T_i &\sim \text{Binomial}\left(p^T_i, n^T_i\right) \\
\text{logit}\left(p^C_i\right) &= \mu_i \\
\text{logit}\left(p^T_i\right) &= \mu_i + \delta \\
\mu_i &\sim \text{Normal}(m, s).
\end{aligned}$$

In this case, we want to make inferences about the population effect $m$, and the predictive distribution for the effect $\delta_{\text{new}}$ in a new trial. 

This particular model uses a random effect for the population mean, and a fixed effect for the treatment effect. There are 3 other models you could fit to represent all possible combinations of fixed or random effects for these two parameters.

Build all 4 models to estimate the treatment effect in PyMC3 and 

1. use convergence diagnostics to check for convergence in each model 
2. use posterior predictive checks to compare the fit of the models
3. use DIC to compare the models as approximations of the true generating model

Which model would you select and why?

In [ ]:
r_t_obs = [3, 7, 5, 102, 28, 4, 98, 60, 25, 138, 64, 45, 9, 57, 25, 33, 28, 8, 6, 32, 27, 22]
n_t_obs = [38, 114, 69, 1533, 355, 59, 945, 632, 278,1916, 873, 263, 291, 858, 154, 207, 251, 151, 174, 209, 391, 680]
r_c_obs = [3, 14, 11, 127, 27, 6, 152, 48, 37, 188, 52, 47, 16, 45, 31, 38, 12, 6, 3, 40, 43, 39]
n_c_obs = [39, 116, 93, 1520, 365, 52, 939, 471, 282, 1921, 583, 266, 293, 883, 147, 213, 122, 154, 134, 218, 364, 674]
N = len(n_c_obs)

In [ ]:
# Write answer here
import pymc3 as pm

niter = 1000
with pm.Model() as meta_analysis:
    
    ###priors on m and s
    m = pm.Normal('m', mu=0.0, sd=1e5, shape=1)
    s = pm.Normal('s', mu=0.0, sd=1e5, shape=1)

    #priors on mu and delta
    mu = pm.Normal('mu', mu=m, sd=s, shape = N)
    delta = pm.Normal('delta', mu=0.0, sd=1e5, shape = 1)

    p_c = pm.math.invlogit(np.array(mu))
    p_t = pm.math.invlogit(np.array(mu+delta))

    r_c = pm.Binomial('r_c', n_c_obs, p_c, observed=r_c_obs)
    r_t = pm.Binomial('r_t', n_t_obs, p_t, observed=r_t_obs)
    
    start = pm.find_MAP()
    trace = pm.sample(1000, start=start, step=pm.Metropolis(), random_seed = 42)
    pm.traceplot(trace)
    